# MPAS unstructured grid data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.tri import Triangulation

import tams

%matplotlib inline

## Load data

In [ ]:
ds = tams.load_example_mpas_ug()
ds

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))

sel = ds.isel(cell=slice(None, None, 20))
ax.scatter(sel.lon, sel.lat, marker=".", s=10, alpha=0.5, edgecolors="none")
ax.set(xlabel="lon", ylabel="lat")
ax.autoscale(tight=True)

In [ ]:
%%time

fig, ax = plt.subplots(figsize=(7.5, 3))

im = ax.scatter(ds.lon, ds.lat, c=ds.tb.isel(time=10), marker=".", s=3, edgecolors="none")
fig.colorbar(im, ax=ax, label="Tb")
ax.autoscale(tight=True)
ax.set(xlabel="lon", ylabel="lat")

In [ ]:
ts = ds.mean("cell", keep_attrs=True)

fig, ax = plt.subplots(figsize=(6, 3))
ax2 = ax.twinx()

ts.tb.plot(ax=ax, c="orangered")
ts.precip.plot(ax=ax2, c="cornflowerblue")
ax.autoscale(axis="x", tight=True)
ax.grid(True)

## Identify CEs

In [ ]:
%%time

itime = 10

x = ds.lon
y = ds.lat
tri = Triangulation(x, y)

In [ ]:
%%time

# Passing the triangulation in is not required but makes it faster
shapes = tams.core._contours_to_gdf(tams.contours(ds.tb.isel(time=itime), value=235, triangulation=tri));

In [ ]:
%%time

cs_ug, cs_ug_core = list(zip(*tams.identify(ds.tb.isel(time=10))))[0]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

tcs = ax.tricontour(tri, ds.tb.isel(time=itime), levels=[235], colors="red", linewidths=1)
tcs_core = ax.tricontour(tri, ds.tb.isel(time=itime), levels=[219], colors="blue", linewidths=1)

shapes.plot(fc="none", ec="green", lw=1.5, ls=":", ax=ax)  # not size-filtered
cs_ug.plot(fc="none", ec="lawngreen", lw=2, ax=ax, zorder=20)

ax.set(xlabel="lon", ylabel="lat")

In [ ]:
ds_rg = tams.load_example_mpas().sel(lat=slice(None, 20))

fig, ax = plt.subplots(figsize=(12, 5))

cs_rg, cs_rg_core = list(zip(*tams.identify(ds_rg.tb.isel(time=itime))))[0]

a = cs_rg_core.plot(fc="none", ec="royalblue", lw=1.5, ls="--", ax=ax)
cs_ug_core.plot(fc="none", ec="mediumblue", lw=2, ax=ax)

cs_rg.plot(fc="none", ec="red", lw=1.5, ls="--", ax=ax)
cs_ug.plot(fc="none", ec="firebrick", lw=2, ax=ax)

ax.set(xlabel="lon", ylabel="lat")